# Chapter 18: Oracles and External Data

---

## 18.1 The Oracle Problem

### 18.1.1 Blockchain Determinism

Blockchains are deterministic systems: given the same input, every node executing a smart contract will produce exactly the same output. This property is essential for consensus. However, it also means that smart contracts cannot directly access external data—such as stock prices, weather information, or random numbers—because that data could vary between nodes or over time.

**The limitation:**
- Smart contracts are isolated from the outside world.
- They can only access data that is already on the blockchain (state, transaction data, block headers).
- To incorporate real-world information, they need a trusted mechanism to bring data in.

### 18.1.2 Need for External Data

Many blockchain applications require external data:

| Use Case | Required Data |
|----------|---------------|
| DeFi lending protocols | Asset prices for collateral valuation |
| Prediction markets | Outcome of events (e.g., election results) |
| Insurance | Weather data, flight delays |
| Gaming | Randomness for loot boxes, game outcomes |
| Supply chain | GPS location, temperature readings |
| Identity | Verifiable credentials from off-chain sources |

Without reliable external data, these applications cannot function. This is where **oracles** come in.

---

## 18.2 Oracle Solutions

An **oracle** is a service that provides external data to a blockchain. Oracles can be centralized (single source) or decentralized (multiple sources with consensus).

### 18.2.1 Centralized Oracles

A centralized oracle is a single entity that pushes data to the blockchain. It's simple and fast but introduces a single point of failure and trust requirement.

**Example: A simple centralized price feed**
```solidity
contract CentralizedPriceFeed {
    address public owner;
    uint256 public price;

    event PriceUpdated(uint256 newPrice);

    constructor() {
        owner = msg.sender;
    }

    function updatePrice(uint256 _price) external {
        require(msg.sender == owner, "Only owner");
        price = _price;
        emit PriceUpdated(_price);
    }

    function getPrice() external view returns (uint256) {
        return price;
    }
}
```

**Problems:**
- Owner can set any price (manipulation risk).
- If owner goes offline, price becomes stale.
- Single point of failure.

### 18.2.2 Decentralized Oracle Networks

Decentralized oracles aggregate data from multiple independent sources, reducing trust in any single party. They use consensus mechanisms to determine the "true" value.

**Key properties:**
- **Multiple nodes**: Data providers (oracles) run by different entities.
- **Aggregation**: The final value is computed from multiple responses (e.g., median, mean).
- **Incentives**: Nodes stake tokens and are rewarded for honest reporting, penalized for dishonesty.
- **Transparency**: All data submissions are on-chain or verifiable off-chain.

**Popular decentralized oracle networks:**
- **Chainlink**: The most widely used, with a network of independent node operators.
- **Tellor**: A decentralized oracle protocol where reporters compete to submit data.
- **API3**: Uses first-party oracles (data providers run their own nodes).

---

## 18.3 Chainlink

Chainlink is the industry standard for decentralized oracles. It provides secure, reliable data feeds for prices, randomness, and custom data requests.

### 18.3.1 Chainlink Architecture

```
Chainlink Architecture:

┌─────────────────────────────────────────────────────────────┐
│                     SMART CONTRACT                          │
│  (Consumer)                                                 │
│  - Requests data                                            │
│  - Receives response                                        │
└───────────────────────────┬─────────────────────────────────┘
                            │
                            ▼
┌─────────────────────────────────────────────────────────────┐
│                 CHAINLINK NETWORK                           │
│  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐         │
│  │   Oracle 1  │  │   Oracle 2  │  │   Oracle 3  │         │
│  └─────────────┘  └─────────────┘  └─────────────┘         │
│  - Fetch data from multiple sources                         │
│  - Aggregate (e.g., median)                                 │
│  - Return result on-chain                                   │
└─────────────────────────────────────────────────────────────┘
                            │
                            ▼
┌─────────────────────────────────────────────────────────────┐
│                  EXTERNAL DATA SOURCES                      │
│  - Binance, Coinbase, Kraken APIs                           │
│  - Weather APIs, sports data, etc.                          │
└─────────────────────────────────────────────────────────────┘
```

Chainlink operates in two primary modes:
1. **Price Feeds**: Pre-aggregated data available via reference contracts (push model).
2. **Request & Receive**: Custom data requests where a contract requests data and Chainlink nodes respond (pull model).

### 18.3.2 Price Feeds Integration

Chainlink Price Feeds are the easiest way to get reliable asset prices. They are updated periodically by a decentralized network.

**Supported assets**: ETH/USD, BTC/USD, LINK/USD, and many others on multiple chains.

**Getting a price feed address**: Check [docs.chain.link](https://docs.chain.link/data-feeds/price-feeds/addresses) for the latest contract addresses per network.

**Example: Using Chainlink Price Feed in Solidity**
```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

import "@chainlink/contracts/src/v0.8/interfaces/AggregatorV3Interface.sol";

contract PriceConsumer {
    AggregatorV3Interface internal priceFeed;

    /**
     * Network: Sepolia
     * Aggregator: ETH/USD
     * Address: 0x694AA1769357215DE4FAC081bf1f309aDC325306
     */
    constructor() {
        priceFeed = AggregatorV3Interface(0x694AA1769357215DE4FAC081bf1f309aDC325306);
    }

    /**
     * Returns the latest price
     */
    function getLatestPrice() public view returns (int) {
        (
            /*uint80 roundID*/,
            int price,
            /*uint startedAt*/,
            /*uint timeStamp*/,
            /*uint80 answeredInRound*/
        ) = priceFeed.latestRoundData();
        return price;
    }

    /**
     * Returns the price with 8 decimals (e.g., 300000000000 = $3000.00000000)
     */
    function getPrice() external view returns (uint) {
        return uint(getLatestPrice());
    }
}
```

**Explanation:**
- `AggregatorV3Interface` defines the interface for price feeds.
- `latestRoundData()` returns the latest round data, including the price.
- Price feeds typically have 8 decimals (except for certain pairs).

### 18.3.3 Verifiable Randomness (VRF)

Chainlink VRF (Verifiable Random Function) provides provably fair random numbers. It's essential for NFTs, gaming, and any application needing unbiased randomness.

**How it works:**
1. Your contract requests randomness by calling `requestRandomness()`.
2. Chainlink VRF generates a random number off-chain and a cryptographic proof.
3. The VRF Coordinator returns the random number and proof to your contract.
4. Your contract receives the random number via a callback function `fulfillRandomness()`.

**Example: Using Chainlink VRF**
```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

import "@chainlink/contracts/src/v0.8/VRFConsumerBase.sol";

contract RandomNumberConsumer is VRFConsumerBase {
    bytes32 internal keyHash;
    uint256 internal fee;
    uint256 public randomResult;

    /**
     * Network: Sepolia
     * Chainlink VRF Coordinator: 0x8103B0A8A00be2DDC778e6e7eaa21791Cd364625
     * LINK Token: 0x01BE23585060835E02B77ef475b0Cc51aA1e0709
     * Key Hash: 0x474e34a077df58807dbe9c96d3c009b23b3c6d0cce433e59bbf5b34f823bc56c
     * Fee: 0.1 LINK
     */
    constructor()
        VRFConsumerBase(
            0x8103B0A8A00be2DDC778e6e7eaa21791Cd364625, // VRF Coordinator
            0x01BE23585060835E02B77ef475b0Cc51aA1e0709  // LINK Token
        )
    {
        keyHash = 0x474e34a077df58807dbe9c96d3c009b23b3c6d0cce433e59bbf5b34f823bc56c;
        fee = 0.1 * 10 ** 18; // 0.1 LINK
    }

    /**
     * Requests randomness
     */
    function getRandomNumber() public returns (bytes32 requestId) {
        require(LINK.balanceOf(address(this)) >= fee, "Not enough LINK");
        return requestRandomness(keyHash, fee);
    }

    /**
     * Callback function used by VRF Coordinator
     */
    function fulfillRandomness(bytes32 requestId, uint256 randomness) internal override {
        randomResult = randomness;
    }
}
```

**Steps to use:**
1. Fund the contract with LINK (Chainlink's native token) to pay for requests.
2. Call `getRandomNumber()`.
3. Wait for the callback; `randomResult` will contain the random number.

### 18.3.4 External API Calls

Chainlink also supports custom API calls through the **Chainlink Any API** or **Chainlink Functions** (newer). This allows smart contracts to fetch any data from the internet.

**Traditional Any API (deprecated in favor of Functions):**
- Contract creates a request specifying URL, path, etc.
- Chainlink nodes fetch the data and return it.

**Chainlink Functions:**
- Write JavaScript code to fetch and process data off-chain.
- Nodes execute the code and return the result on-chain.
- More flexible and cheaper.

**Example: Chainlink Functions request**
```javascript
// JavaScript code executed off-chain
const apiResponse = await Functions.makeHttpRequest({
  url: "https://api.coingecko.com/api/v3/simple/price?ids=ethereum&vs_currencies=usd"
});
if (apiResponse.error) {
  throw new Error("Request failed");
}
const { data } = apiResponse;
return Functions.encodeUint256(Math.round(data.ethereum.usd * 100));
```

**Solidity contract using Functions:**
```solidity
import {Functions, FunctionsClient} from "@chainlink/contracts/src/v0.8/functions/v1_0_0/FunctionsClient.sol";
import {ConfirmedOwner} from "@chainlink/contracts/src/v0.8/shared/access/ConfirmedOwner.sol";

contract APIConsumer is FunctionsClient, ConfirmedOwner {
    using Functions for Functions.Request;

    bytes32 public latestRequestId;
    bytes public latestResponse;
    bytes public latestError;

    constructor(address oracle) FunctionsClient(oracle) ConfirmedOwner(msg.sender) {}

    function executeRequest(
        string calldata source,
        bytes calldata secrets,
        Functions.Location secretsLocation,
        string[] calldata args,
        uint64 subscriptionId,
        uint32 gasLimit
    ) public onlyOwner {
        Functions.Request memory req;
        req.initializeRequest(Functions.Location.Inline, Functions.CodeLanguage.JavaScript, source);
        if (secrets.length > 0) {
            if (secretsLocation == Functions.Location.Inline) {
                req.addInlineSecrets(secrets);
            } else {
                req.addRemoteSecrets(secrets);
            }
        }
        if (args.length > 0) req.addArgs(args);

        latestRequestId = _sendRequest(req.encodeCBOR(), subscriptionId, gasLimit);
    }

    function fulfillRequest(bytes32 requestId, bytes memory response, bytes memory err) internal override {
        latestResponse = response;
        latestError = err;
    }
}
```

Chainlink Functions is the recommended approach for custom data fetching.

---

## 18.4 Code Implementation: Using Oracles

Let's build a practical example: a contract that allows users to bet on the price of ETH, using Chainlink Price Feeds for settlement.

### 18.4.1 Integrating Chainlink Price Feeds

We'll create a simple betting contract:
- Users place bets on whether ETH price will be above or below a target at a future time.
- The contract uses Chainlink to get the price at expiration.
- Winners receive the pot.

```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

import "@chainlink/contracts/src/v0.8/interfaces/AggregatorV3Interface.sol";

contract PriceBet {
    enum Direction { Above, Below }
    enum BetState { Open, Closed, Settled }

    struct Bet {
        address payable better;
        Direction direction;
        uint256 amount;
        uint256 targetPrice;
        uint256 deadline;
        BetState state;
    }

    Bet[] public bets;
    AggregatorV3Interface internal priceFeed;

    event BetPlaced(uint indexed betId, address indexed better, Direction direction, uint256 amount, uint256 targetPrice, uint256 deadline);
    event BetSettled(uint indexed betId, address indexed better, uint256 payout, bool won);

    constructor(address _priceFeed) {
        priceFeed = AggregatorV3Interface(_priceFeed);
    }

    function placeBet(Direction _direction, uint256 _targetPrice, uint256 _duration) external payable {
        require(msg.value > 0, "Must send ETH");
        require(_targetPrice > 0, "Invalid target");

        uint256 deadline = block.timestamp + _duration;
        bets.push(Bet({
            better: payable(msg.sender),
            direction: _direction,
            amount: msg.value,
            targetPrice: _targetPrice,
            deadline: deadline,
            state: BetState.Open
        }));

        emit BetPlaced(bets.length - 1, msg.sender, _direction, msg.value, _targetPrice, deadline);
    }

    function settleBet(uint _betId) external {
        Bet storage bet = bets[_betId];
        require(bet.state == BetState.Open, "Bet not open");
        require(block.timestamp >= bet.deadline, "Bet not expired");

        // Get current price from Chainlink
        (, int price, , , ) = priceFeed.latestRoundData();
        uint256 currentPrice = uint256(price); // assuming 8 decimals

        bool won;
        if (bet.direction == Direction.Above) {
            won = currentPrice > bet.targetPrice;
        } else {
            won = currentPrice < bet.targetPrice;
        }

        bet.state = BetState.Settled;

        if (won) {
            // Winner gets the pot (simplified: returns their bet, but in reality would distribute all losers' funds)
            bet.better.transfer(bet.amount);
            emit BetSettled(_betId, bet.better, bet.amount, true);
        } else {
            // Loser loses their bet (contract retains funds)
            emit BetSettled(_betId, bet.better, 0, false);
        }
    }

    // Allow contract to receive ETH
    receive() external payable {}
}
```

This simplified example only returns the bet amount to winners; in a real contract, you'd pool all bets and redistribute. But it shows how to integrate a price feed.

### 18.4.2 Request-Response Pattern

For custom data (e.g., weather, sports scores), we use the request-response pattern. Here's a minimal example using Chainlink Any API (conceptual; use Functions for production).

```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

import "@chainlink/contracts/src/v0.8/ChainlinkClient.sol";

contract WeatherOracle is ChainlinkClient {
    using Chainlink for Chainlink.Request;

    uint256 public temperature;
    string public city;
    address private oracle;
    bytes32 private jobId;
    uint256 private fee;

    event RequestTemperature(bytes32 indexed requestId, string city);
    event TemperatureReceived(bytes32 indexed requestId, uint256 temperature);

    constructor(address _oracle, bytes32 _jobId, uint256 _fee, address _link) {
        setChainlinkToken(_link);
        oracle = _oracle;
        jobId = _jobId;
        fee = _fee;
    }

    function requestTemperature(string memory _city) public {
        city = _city;
        Chainlink.Request memory req = buildChainlinkRequest(jobId, address(this), this.fulfill.selector);
        req.add("get", string(abi.encodePacked("https://api.openweathermap.org/data/2.5/weather?q=", _city, "&appid=YOUR_API_KEY")));
        req.add("path", "main.temp"); // JSON path
        req.addInt("times", 100); // multiply by 100 to handle decimal
        sendChainlinkRequestTo(oracle, req, fee);
        emit RequestTemperature(keccak256(abi.encodePacked(_city)), _city);
    }

    function fulfill(bytes32 _requestId, uint256 _temperature) public recordChainlinkFulfillment(_requestId) {
        temperature = _temperature;
        emit TemperatureReceived(_requestId, _temperature);
    }
}
```

### 18.4.3 Best Practices

1. **Use multiple sources**: For critical data, aggregate from multiple oracles.
2. **Handle stale data**: Check timestamp from `latestRoundData()` to ensure data is recent.
3. **Fallback mechanisms**: If primary oracle fails, have a backup.
4. **Gas considerations**: Oracle calls are expensive; optimize where possible.
5. **Test thoroughly**: Use testnets and mock oracles.

---

## Chapter Summary

```
┌─────────────────────────────────────────────────────────────────┐
│                    CHAPTER 18 SUMMARY                           │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  • The Oracle Problem: Blockchains cannot access external data │
│    natively; oracles bridge the gap.                           │
│                                                                 │
│  • Centralized oracles are simple but trust-based.             │
│    Decentralized oracle networks (Chainlink, Tellor)           │
│    provide trust-minimized data.                               │
│                                                                 │
│  • Chainlink Price Feeds:                                      │
│    - Pre-aggregated price data for many assets.                │
│    - Easy integration via AggregatorV3Interface.               │
│    - Used for DeFi, lending, derivatives.                      │
│                                                                 │
│  • Chainlink VRF:                                              │
│    - Provably fair random numbers.                             │
│    - Essential for gaming, NFTs, lotteries.                    │
│    - Requires LINK payment.                                    │
│                                                                 │
│  • Chainlink Functions:                                        │
│    - Custom API calls with JavaScript off-chain.               │
│    - Flexible, cost-effective.                                 │
│                                                                 │
│  • Security best practices:                                    │
│    - Check data freshness.                                     │
│    - Use multiple sources if possible.                         │
│    - Test with mock oracles.                                   │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Next Chapter Preview:** Chapter 19 – Token Standards. We'll dive deep into ERC-20, ERC-721, ERC-1155, and other important token standards, with complete implementation examples and best practices.